# Problem 1: (50% for CSC 446, 50% for CSC 546)

To ensure that you read and fully understand the template source code for discrete-event simulation of single-queue-single-server systerm, translate the sample Jave code posted in Brightspace--Content--Weeks 2,3 to Python. **Note that you are not allowed to use existing python function to generate exponential or normal variates.**

Using the given default parameters given in the sample code, generate 400 customers, and record their arrival times, departure times, service times, and waiting times in the queue in the following format:

| Customer #  | Arrival Time      |  Departure Time | Service Time for the Customer| The Customer's Waiting Time in the Queue|
|:---------------------:|:------------:|:-------------|:----------:|:-----|






In [34]:
''' Write your Python source code is below
'''
import random
import math
class Event:
    def __init__ (self,a_type,a_time):
        self._type = a_type
        self.time = a_time
        self.leftlink = None
        self.rightlink = None
        self.uplink = None
        
    def get_type(self):
        return self._type
    def get_time(self):
        return self.time
    def __lt__(self, other):
        return self.time < other.get_time()

    def __eq__(self, other):
        return self.time == other.get_time()

    def __gt__(self, other):
        return self.time > other.get_time()
    # def compareTo(self,other):
    #     if self.time<other.get_time():
    #         return -1
    #     elif self.time == other.get_time():
    #         return 0
    #     else:
    #         return 1

# splay tree from :https://github.com/anoopj/pysplay/blob/master/splay.py

class Node:
    def __init__(self, key):
        self.key = key
        self.left = self.right = None

    def equals(self, node):
        return self.key == node.key

class SplayTree:
    def __init__(self):
        self.root = None
        self.header = Node(None) #For splay()

    def insert(self, key):
        if (self.root == None):
            self.root = Node(key)
            return

        self.splay(key)
        if self.root.key == key:
            # If the key is already there in the tree, don't do anything.
            return

        n = Node(key)
        if key < self.root.key:
            n.left = self.root.left
            n.right = self.root
            self.root.left = None
        else:
            n.right = self.root.right
            n.left = self.root
            self.root.right = None
        self.root = n

    def remove(self, key):
        self.splay(key)
        if key != self.root.key:
            raise 'key not found in tree'

        # Now delete the root.
        if self.root.left== None:
            self.root = self.root.right
        else:
            x = self.root.right
            self.root = self.root.left
            self.splay(key)
            self.root.right = x

    def findMin(self):
        if self.root == None:
            return None
        x = self.root
        while x.left != None:
            x = x.left
        self.splay(x.key)
        return x.key

    def findMax(self):
        if self.root == None:
            return None
        x = self.root
        while (x.right != None):
            x = x.right
        self.splay(x.key)
        return x.key

    def find(self, key):
        if self.root == None:
            return None
        self.splay(key)
        if self.root.key != key:
            return None
        return self.root.key

    def isEmpty(self):
        return self.root == None
    
    def splay(self, key):
        l = r = self.header
        t = self.root
        self.header.left = self.header.right = None
        while True:
            if key < t.key:
                if t.left == None:
                    break
                if key < t.left.key:
                    y = t.left
                    t.left = y.right
                    y.right = t
                    t = y
                    if t.left == None:
                        break
                r.left = t
                r = t
                t = t.left
            elif key > t.key:
                if t.right == None:
                    break
                if key > t.right.key:
                    y = t.right
                    t.right = y.left
                    y.left = t
                    t = y
                    if t.right == None:
                        break
                l.right = t
                l = t
                t = t.right
            else:
                break
        l.right = t.left
        r.left = t.right
        t.left = self.header.right
        t.right = self.header.left
        self.root = t


random.seed(10) 
customer_size = 400
MeanIAT = 4.5  
MeanST = 3.2
sigma = 0.6
PI = 3.1415927

arrival = 1 
departure = 2

clock = 0.0
num_in_service = 0
queue_len = 0
max_queue_len = 0
total_busy = 0
last_event_time = 0

sum_response_time = 0
num_depart = 0
long_service = 0

num_normals = 0
save_normal = 0
event_list = SplayTree()
cust_list = []
customer = [i for i in range(customer_size)]
arrival_time = []
service_time = []
depart_time = []

def generate_exp(MeanIAT):
    u = random.random()
    exponential_variate = - MeanIAT *  math.log(u)
    return exponential_variate

def normal(MeanST, sigma):
    global num_normals,save_normal
    
    if num_normals == 0:
        r1 = random.random()
        r2 = random.random()

        return_normal = math.sqrt(-2 * math.log(r1)) * math.cos(2 * PI * r2)
        save_normal = math.sqrt(-2 * math.log(r1)) * math.sin(2 * PI * r2)
        num_normals = 1
    else:
        num_normals = 0
        return_normal = save_normal

    return return_normal * sigma + MeanST

event = Event(arrival, generate_exp(MeanIAT))
event_list.insert(event)
arrival_time.append(event.time)

def process_arrival(event):
    global queue_len,total_busy,last_event_time,max_queue_len
    queue_len += 1
    cust_list.append(event)
    if num_in_service == 0:
        schedule_departure()
    else:
        total_busy += (clock - last_event_time)
    if max_queue_len < queue_len:
        max_queue_len = queue_len
    
    next_arrival = Event(arrival,clock + generate_exp(MeanIAT))
    arrival_time.append(next_arrival.time)
    event_list.insert(next_arrival)
    last_event_time = clock

def schedule_departure():
    global queue_len, num_in_service
    ServiceTime = 0
    while ServiceTime<=0:
        ServiceTime = normal(MeanST, sigma)
    service_time.append(ServiceTime)
    
    num_in_service = 1
    queue_len -= 1
    depart = Event(departure,clock+ServiceTime)
    event_list.insert(depart)
    depart_time.append(depart.time)
    

def process_depart(event):
    global total_busy,queue_len,num_depart,num_in_service,clock,last_event_time,sum_response_time,long_service

    departure = cust_list.pop()
    if queue_len > 0:
        schedule_departure()        
    else:
        num_in_service = 0

    response = clock - departure.time
    sum_response_time += response
    long_service+= response > 4.0

    total_busy += clock - last_event_time
    num_depart += 1
    last_event_time = clock


while num_depart < customer_size:
    event = event_list.findMin()
    event_list.remove(event)
    clock = event.time
    process_arrival(event) if event.get_type() == arrival else process_depart(event)

print("SINGLE SERVER QUEUE SIMULATION - GROCERY STORE CHECKOUT COUNTER ")
print("MEAN INTERARRIVAL TIME                         ",MeanIAT)
print( "MEAN SERVICE TIME                              ",MeanST )
print( "STANDARD DEVIATION OF SERVICE TIMES            ",sigma )
print ("NUMBER OF CUSTOMERS SERVED                     ",customer_size)
print( "SERVER UTILIZATION                             ",total_busy/clock )
print( "MAXIMUM LINE LENGTH                            ",max_queue_len )
print( "AVERAGE RESPONSE TIME                          ",sum_response_time/customer_size,"  MINUTES" )
print( "PROPORTION WHO SPEND FOUR ")
print( "MINUTES OR MORE IN SYSTEM                     ",long_service/customer_size)
print( "SIMULATION RUNLENGTH                           ",clock," MINUTES" )
print( "NUMBER OF DEPARTURES                           ",customer_size )

customer = [i for i in range(1,customer_size+1)]

Time_Service_Begin = [0 for i in range(customer_size)]
Time_Customer_Waiting_in_Queue = [0 for i in range(customer_size)]
Time_Service_Ends = [0 for i in range(customer_size)]
Time_Customer_Spend_in_System = [0 for i in range(customer_size)]
System_idle = [0 for i in range(customer_size)]

Time_Service_Begin[0] = arrival_time[0]
Time_Service_Ends[0] = service_time[0]
Time_Customer_Spend_in_System[0] = service_time[0]
Time_Service_Ends[0] = Time_Service_Begin[0] + service_time[0]

for i in range(1,customer_size):
  # Time Service Begin 
  Time_Service_Begin[i] = max(arrival_time[i],Time_Service_Ends[i-1])
    
  # Time customer waiting in queue   
  Time_Customer_Waiting_in_Queue[i] = Time_Service_Begin[i]-arrival_time[i]

  # Time service ends
  Time_Service_Ends[i] = Time_Service_Begin[i] + service_time[i]  

  # Time Customer Spend in the system
  Time_Customer_Spend_in_System[i] = Time_Service_Ends[i] - arrival_time[i]
    

from prettytable import PrettyTable
while(len(arrival_time)>customer_size):
  arrival_time.pop(-1)
while(len(service_time)>customer_size):
  service_time.pop(-1)
x = PrettyTable()

column_names =['Customer #','Arrival Time','Departure Time','Service Time for the Customer','The Customer\'s Waiting Time in the Queue']
data = [customer,arrival_time,Time_Service_Ends,service_time,Time_Customer_Waiting_in_Queue]
length = len(column_names)

for i in range(length):
  x.add_column(column_names[i],data[i])
  
print(x)




SINGLE SERVER QUEUE SIMULATION - GROCERY STORE CHECKOUT COUNTER 
MEAN INTERARRIVAL TIME                          4.5
MEAN SERVICE TIME                               3.2
STANDARD DEVIATION OF SERVICE TIMES             0.6
NUMBER OF CUSTOMERS SERVED                      400
SERVER UTILIZATION                              0.729932262635987
MAXIMUM LINE LENGTH                             11
AVERAGE RESPONSE TIME                           8.9847122287616   MINUTES
PROPORTION WHO SPEND FOUR 
MINUTES OR MORE IN SYSTEM                      0.345
SIMULATION RUNLENGTH                            1739.1476878041162  MINUTES
NUMBER OF DEPARTURES                            400


<class 'ModuleNotFoundError'>: No module named 'prettytable'

# End of Problem 1

# Problem 2 (50% for CSC 446, 25% for CSC 546)

1. Apply the Chi-square test to the recorded 400 customers in Problem 1 to test the hypothesis that the customer arrival
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: The customer arrival process follows a Poisson process since the value of p is greater than 0.05.

2. Apply the Chi-square test to the recorded 400 customers in Problem 1 to test the hypothesis that the customer depature
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: The customer depature process does not follow a Poisson process, since the value of p is less than 0.05.

3. What is your conclusion after the statistical test?

Ans: Since the service time follows a normal distribution, The customer depature process does not follow a Poisson process; However, the customer arrival process follows a Poisson process.

**Hint: You can use inter-arrival times test for Poisson process: You can examine the inter-arrival times between successive events and check if they follow an exponential distribution. If the inter-arrival times follow an exponential distribution, then the arrival process can be considered a Poisson process.**


# End of Problem 2


In [25]:
#2-a
import scipy.stats as stats 
import numpy as np
import math

k = int(math.sqrt(customer_size))
p = 1/k
_lambda = 1/MeanIAT 


IAT = [0 for i in range (customer_size)]
IAT[0] = arrival_time[0]
for i in range(1,customer_size):
    IAT[i] = arrival_time[i] - IAT[i - 1]
    
endpoints = [(-1 / _lambda) * math.log(1 - i * p) for i in range(k)]
observed = np.histogram(IAT, bins=endpoints + [np.inf])[0]
expected = np.full(k, customer_size / k)

chi2_stat, p_value = stats.chisquare(observed, f_exp=expected)

print("Expected counts:", expected)
print("Observed counts:", observed)
print("Chi-square statistic:", chi2_stat)
print("p-value:", p_value)


<class 'IndexError'>: list index out of range

In [24]:
#2-b
import math
import scipy.stats as stats
import numpy as np

MeanInterdepartureTime = 0
customer_size = len(Time_Service_Ends)
inter_departure_time = [Time_Service_Ends[0]] + [Time_Service_Ends[i] - Time_Service_Ends[i - 1] for i in range(1, customer_size)]

for i in range(1,customer_size):
    MeanInterdepartureTime += inter_departure_time[i]

MeanInterdepartureTime = MeanInterdepartureTime/len(inter_departure_time)

k = int(math.sqrt(TotalCustomers))
p = 1 / k
lam = 1 / 4.5

endpoints = [(-1 / lam) * math.log(1 - i * p) for i in range(k)]
observed = np.histogram(inter_departure_time, bins=endpoints + [np.inf])[0]
expected = np.full(k, TotalCustomers / k)

chi2_stat, p_value = stats.chisquare(observed, f_exp=expected)

print("Chi-square statistic:", chi2_stat)
print("p-value:", p_value)


0.0
Chi-square statistic: 7600.0
p-value: 0.0


# Problem 3 (CSC 546 only, 25% for CSC 546)

Change the service times to exponentially distributed with mean value of 3.2. Run your simulation to generate 400 customers, and and record their arrival times, departure times, service times, and waiting times using the same format as in Problem 1.


1. Apply the Chi-square test to the recorded customers to test the hypothesis that the customer arrival
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here. 

2. Apply the Chi-square test to the recorded customers to test the hypothesis that the customer depature
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here. 


3. What is your conclusion after the statistical test?

Ans: Write your answer here. 
**Hint: You can use inter-arrival times test for Poisson process: You can examine the inter-arrival times between successive events and check if they follow an exponential distribution. If the inter-arrival times follow an exponential distribution, then the arrival process can be considered a Poisson process.**

# End of Problem 3
